<a href="https://colab.research.google.com/github/farshid101/Thesis-2024/blob/main/llam/llama2_example_kor_nli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QLoRAÎ•º ÌôúÏö©Ìïú ÌïúÍµ≠Ïñ¥ LLM Fine-Tuning: Train LLaMA2 7B as a Text Classifier

## 0. ÎùºÏù¥Î∏åÎü¨Î¶¨ ÏÑ§Ïπò

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install trl

   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 122.4/122.4 MB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 3.0/3.0 MB 34.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 472.7/472.7 kB 17.0 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ

## 1. Îç∞Ïù¥ÌÑ∞ÏÖã Ï§ÄÎπÑ

- KLUEÏùò NLI Îç∞Ïù¥ÌÑ∞ÏÖãÏùÑ ÏÇ¨Ïö©Ìï©ÎãàÎã§.

In [2]:
from datasets import load_dataset, Dataset, DatasetDict
from dataclasses import dataclass, field
from typing import Optional
import torch
from peft import LoraConfig
from tqdm import tqdm
import pandas as pd
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, AutoTokenizer, pipeline
from trl import SFTTrainer

tqdm.pandas()

nli_data = load_dataset('klue', 'nli')

train_data = nli_data['train']
validation_data = nli_data['validation']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/224k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24998 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [14]:
# Convert to DataFrame for easier viewing
train_df = pd.DataFrame(train_data)
train_df.head()  # Show the first few rows of the DataFrame


guid  source                   premise  \
0  klue-nli-v1_train_00000    NSMC  ÌûõÍ±∏ ÏßÑÏã¨ ÏµúÍ≥†Îã§ Í∑∏ Ïñ¥Îñ§ ÌûàÏñ¥Î°úÎ≥¥Îã§ Î©ãÏßÄÎã§   
1  klue-nli-v1_train_00001    NSMC   100Î∂ÑÍ∞Ñ ÏûòÍªÑ Í∑∏ÎûòÎèÑ ÏÜåÎãâÎ∂êÎïúÏóê 2Ï†êÏ§ÄÎã§   
2  klue-nli-v1_train_00002    NSMC   100Î∂ÑÍ∞Ñ ÏûòÍªÑ Í∑∏ÎûòÎèÑ ÏÜåÎãâÎ∂êÎïúÏóê 2Ï†êÏ§ÄÎã§   
3  klue-nli-v1_train_00003    NSMC   100Î∂ÑÍ∞Ñ ÏûòÍªÑ Í∑∏ÎûòÎèÑ ÏÜåÎãâÎ∂êÎïúÏóê 2Ï†êÏ§ÄÎã§   
4  klue-nli-v1_train_00004  airbnb  101ÎπåÎî© Í∑ºÏ≤òÏóê ÎÇòÎ¶Ñ Ï¶êÍ∏∏Í±∞Î¶¨Í∞Ä ÎßéÏäµÎãàÎã§.   

                   hypothesis  label  
0              ÌûõÍ±∏ ÏßÑÏã¨ ÏµúÍ≥†Î°ú Î©ãÏßÄÎã§.      0  
1                   100Î∂ÑÍ∞Ñ Ïû§Îã§.      2  
2               ÏÜåÎãâÎ∂êÏù¥ Ï†ïÎßê Î©ãÏûàÏóàÎã§.      1  
3       100Î∂ÑÍ∞Ñ ÏûêÎäîÍ≤å Îçî ÎÇòÏïòÏùÑ Í≤É Í∞ôÎã§.      1  
4  101ÎπåÎî© Í∑ºÏ≤òÏóêÏÑú Ï¶êÍ∏∏Í±∞Î¶¨ Ï∞æÍ∏∞Îäî Ïñ¥Î†µÏäµÎãàÎã§.      2

In [15]:
# Convert to DataFrame for easier viewing
validation_data = pd.DataFrame(validation_data)
validation_data.head()  # Show the first few rows of the DataFrame


guid  source                              premise  \
0  klue-nli-v1_dev_00000  airbnb  Ìù°Ïó∞ÏûêÎ∂ÑÎì§ÏùÄ Î∞úÏΩîÎãàÍ∞Ä ÏûàÎäî Î∞©Ïù¥Î©¥ Î∞úÏΩîÎãàÏóêÏÑú Ìù°Ïó∞Ïù¥ Í∞ÄÎä•Ìï©ÎãàÎã§.   
1  klue-nli-v1_dev_00001  airbnb             10Î™ÖÏù¥ Ìï®Íªò ÏÇ¨Ïö©ÌïòÍ∏∞ Î∂àÌé∏Ìï®ÏóÜÏù¥ ÎßåÏ°±ÌñàÎã§.   
2  klue-nli-v1_dev_00002  airbnb             10Î™ÖÏù¥ Ìï®Íªò ÏÇ¨Ïö©ÌïòÍ∏∞ Î∂àÌé∏Ìï®ÏóÜÏù¥ ÎßåÏ°±ÌñàÎã§.   
3  klue-nli-v1_dev_00003  airbnb             10Î™ÖÏù¥ Ìï®Íªò ÏÇ¨Ïö©ÌïòÍ∏∞ Î∂àÌé∏Ìï®ÏóÜÏù¥ ÎßåÏ°±ÌñàÎã§.   
4  klue-nli-v1_dev_00004  airbnb     10Ï∏µÏóê Í±¥Î¨ºÏÇ¨ÎûåÎì§Îßå Ïù¥Ïö©ÌïòÎäî ÏàòÏòÅÏû•Í≥º Ïç¨Î≤†ÎìúÎì§Ïù¥ ÏûàÍµ¨Ïöî.   

                     hypothesis  label  
0            Ïñ¥Îñ§ Î∞©ÏóêÏÑúÎèÑ Ìù°Ïó∞ÏùÄ Í∏àÏßÄÎê©ÎãàÎã§.      2  
1        10Î™ÖÏù¥ Ìï®Íªò ÏÇ¨Ïö©ÌïòÍ∏∞ Î∂àÌé∏Ìï®Ïù¥ ÎßéÏïòÎã§.      2  
2    ÏÑ±Ïù∏ 10Î™ÖÏù¥ Ìï®Íªò ÏÇ¨Ïö©ÌïòÍ∏∞ Î∂àÌé∏Ìï®ÏóÜÏù¥ ÏóÜÏóàÎã§.      1  
3         10Î™ÖÏù¥ Ìï®Íªò ÏÇ¨Ïö©ÌïòÍ∏∞Ïóê ÎßåÏ°±Ïä§Îü¨Ïõ†Îã§.      0  
4  Í±¥Î¨ºÏÇ¨ÎûåÎì§ÏùÄ ÏàòÏòÅÏû•Í≥º Ïç¨Î≤†ÎìúÎ•º Ïù¥Ïö©Ìï† Ïàò ÏûàÏäµÎãàÎã§.      0

# Adding my Dataset


In [16]:
df = pd.read_csv('/content/Final_clean pro 2.csv')
df.head()

Comments   Targets
0  ‡¶§‡ßã‡¶Æ‡¶∞‡¶æ ‡¶∏‡¶¨‡¶æ‡¶á ‡¶è‡¶á ‡¶≠‡¶æ‡¶á ‡¶§‡¶æ‡¶ï‡ßá ‡¶è‡¶ï‡¶ü‡ßÅ  ‡¶ï‡ßã‡¶∞‡ßã‡¶≠‡¶ø‡¶°‡¶ø‡¶ì ‡¶ó‡ßÅ‡¶≤‡ßã ‡¶¶‡ßá...  positive
1  üéØ ‡¶¨‡ßç‡¶∞‡¶π‡ßç‡¶Æ‡¶™‡ßÅ‡¶§‡ßç‡¶∞ ‡¶®‡¶¶‡ßá‡¶∞ ‡¶ì‡¶™‡¶∞ ‡¶®‡¶ø‡¶∞‡ßç‡¶Æ‡¶ø‡¶§ ‡ßØ.‡ßß‡ß´ ‡¶ï‡¶ø‡¶≤‡ßã‡¶Æ‡¶ø‡¶ü‡¶æ‡¶∞‡ßá...  positive
2         üòä ‡¶ß‡¶®‡ßç‡¶Ø‡¶¨‡¶æ‡¶¶ ‡¶Æ‡¶æ‡¶®‡¶®‡ßÄ‡¶Ø‡¶º ‡¶™‡ßç‡¶∞‡¶ß‡¶æ‡¶®‡¶Æ‡¶®‡ßç‡¶§‡ßç‡¶∞‡ßÄ ‡¶∂‡ßá‡¶ñ ‡¶π‡¶æ‡¶∏‡¶ø‡¶®‡¶æ  positive
3                                2022 ‡¶è‡¶∞ 25 ‡¶è ‡¶ú‡ßÅ‡¶® ü•∞ü•∞   neutral
4  ‡¶Ö‡¶®‡ßÅ‡¶≠‡ßÇ‡¶§‡¶ø‡¶ü‡¶æ ‡¶è‡¶§ ‡¶∏‡ßÅ‡¶®‡ßç‡¶¶‡¶∞ ‡¶≠‡¶æ‡¶∑‡¶æ‡¶Ø‡¶º ‡¶™‡ßç‡¶∞‡¶ï‡¶æ‡¶∂ ‡¶ï‡¶∞‡¶§‡ßá ‡¶™‡¶æ‡¶∞‡¶§‡ßá‡¶õ‡¶ø...  positive

In [18]:
import pandas as pd

# Function to rename a single column
def change_column_name(df, old_name, new_name):
    df = df.rename(columns={old_name: new_name})
    return df

# Assuming df is your DataFrame
df = change_column_name(df, old_name="Comments", new_name="hypothesis")
df = change_column_name(df, old_name="Targets", new_name="label")
df.head()  # Display the first few rows of the DataFrame
#text

hypothesis     label
0  ‡¶§‡ßã‡¶Æ‡¶∞‡¶æ ‡¶∏‡¶¨‡¶æ‡¶á ‡¶è‡¶á ‡¶≠‡¶æ‡¶á ‡¶§‡¶æ‡¶ï‡ßá ‡¶è‡¶ï‡¶ü‡ßÅ  ‡¶ï‡ßã‡¶∞‡ßã‡¶≠‡¶ø‡¶°‡¶ø‡¶ì ‡¶ó‡ßÅ‡¶≤‡ßã ‡¶¶‡ßá...  positive
1  üéØ ‡¶¨‡ßç‡¶∞‡¶π‡ßç‡¶Æ‡¶™‡ßÅ‡¶§‡ßç‡¶∞ ‡¶®‡¶¶‡ßá‡¶∞ ‡¶ì‡¶™‡¶∞ ‡¶®‡¶ø‡¶∞‡ßç‡¶Æ‡¶ø‡¶§ ‡ßØ.‡ßß‡ß´ ‡¶ï‡¶ø‡¶≤‡ßã‡¶Æ‡¶ø‡¶ü‡¶æ‡¶∞‡ßá...  positive
2         üòä ‡¶ß‡¶®‡ßç‡¶Ø‡¶¨‡¶æ‡¶¶ ‡¶Æ‡¶æ‡¶®‡¶®‡ßÄ‡¶Ø‡¶º ‡¶™‡ßç‡¶∞‡¶ß‡¶æ‡¶®‡¶Æ‡¶®‡ßç‡¶§‡ßç‡¶∞‡ßÄ ‡¶∂‡ßá‡¶ñ ‡¶π‡¶æ‡¶∏‡¶ø‡¶®‡¶æ  positive
3                                2022 ‡¶è‡¶∞ 25 ‡¶è ‡¶ú‡ßÅ‡¶® ü•∞ü•∞   neutral
4  ‡¶Ö‡¶®‡ßÅ‡¶≠‡ßÇ‡¶§‡¶ø‡¶ü‡¶æ ‡¶è‡¶§ ‡¶∏‡ßÅ‡¶®‡ßç‡¶¶‡¶∞ ‡¶≠‡¶æ‡¶∑‡¶æ‡¶Ø‡¶º ‡¶™‡ßç‡¶∞‡¶ï‡¶æ‡¶∂ ‡¶ï‡¶∞‡¶§‡ßá ‡¶™‡¶æ‡¶∞‡¶§‡ßá‡¶õ‡¶ø...  positive

In [19]:
df.value_counts('label')

label
positive    2713
negative    1895
neutral     1767
Name: count, dtype: int64

In [21]:
df['label'] = df.label.map({'positive': 1, 'neutral': 0, 'negative': 2})
df.head()

hypothesis  label
0  ‡¶§‡ßã‡¶Æ‡¶∞‡¶æ ‡¶∏‡¶¨‡¶æ‡¶á ‡¶è‡¶á ‡¶≠‡¶æ‡¶á ‡¶§‡¶æ‡¶ï‡ßá ‡¶è‡¶ï‡¶ü‡ßÅ  ‡¶ï‡ßã‡¶∞‡ßã‡¶≠‡¶ø‡¶°‡¶ø‡¶ì ‡¶ó‡ßÅ‡¶≤‡ßã ‡¶¶‡ßá...      1
1  üéØ ‡¶¨‡ßç‡¶∞‡¶π‡ßç‡¶Æ‡¶™‡ßÅ‡¶§‡ßç‡¶∞ ‡¶®‡¶¶‡ßá‡¶∞ ‡¶ì‡¶™‡¶∞ ‡¶®‡¶ø‡¶∞‡ßç‡¶Æ‡¶ø‡¶§ ‡ßØ.‡ßß‡ß´ ‡¶ï‡¶ø‡¶≤‡ßã‡¶Æ‡¶ø‡¶ü‡¶æ‡¶∞‡ßá...      1
2         üòä ‡¶ß‡¶®‡ßç‡¶Ø‡¶¨‡¶æ‡¶¶ ‡¶Æ‡¶æ‡¶®‡¶®‡ßÄ‡¶Ø‡¶º ‡¶™‡ßç‡¶∞‡¶ß‡¶æ‡¶®‡¶Æ‡¶®‡ßç‡¶§‡ßç‡¶∞‡ßÄ ‡¶∂‡ßá‡¶ñ ‡¶π‡¶æ‡¶∏‡¶ø‡¶®‡¶æ      1
3                                2022 ‡¶è‡¶∞ 25 ‡¶è ‡¶ú‡ßÅ‡¶® ü•∞ü•∞      0
4  ‡¶Ö‡¶®‡ßÅ‡¶≠‡ßÇ‡¶§‡¶ø‡¶ü‡¶æ ‡¶è‡¶§ ‡¶∏‡ßÅ‡¶®‡ßç‡¶¶‡¶∞ ‡¶≠‡¶æ‡¶∑‡¶æ‡¶Ø‡¶º ‡¶™‡ßç‡¶∞‡¶ï‡¶æ‡¶∂ ‡¶ï‡¶∞‡¶§‡ßá ‡¶™‡¶æ‡¶∞‡¶§‡ßá‡¶õ‡¶ø...      1

In [27]:
df["premise"]=df["hypothesis"]

# Connecting data to my dataset


In [28]:
df.count()

hypothesis    6375
label         6375
premise       6375
dtype: int64

In [29]:
train_data =df[:4000]
validation_data = df[4005:5000]

In [35]:
# Convert to DataFrame for easier viewing
validation_data = pd.DataFrame(validation_data)
validation_data.head()  # Show the first few rows of the DataFrame

hypothesis  label  \
4005                        ‡¶®‡¶æ‡¶∂‡¶ï‡¶§‡¶æ‡¶ï‡¶æ‡¶∞‡ßÄ ‡¶ï‡ßá?‡¶¶‡ßá‡¶∂‡ßá‡¶∞ ‡¶ú‡¶®‡¶ó‡¶£ ‡¶ï‡ßá      0   
4006                              ‡¶§‡ßã‡¶∞ ‡¶¨‡¶ø‡¶ö‡¶æ‡¶∞ ‡¶Ü‡¶≤‡ßç‡¶≤‡¶æ‡¶π ‡¶ï‡¶∞‡¶¨‡ßá      1   
4007           ‡¶ï‡¶æ‡¶∞‡ßã ‡¶ö‡ßã‡¶ñ‡ßá‡¶∞ ‡¶™‡¶æ‡¶®‡¶ø ‡¶™‡¶°‡¶º‡ßá‡¶®‡¶ø ‡¶¨‡¶∞‡¶Ç ‡¶ú‡¶®‡¶ó‡¶£ ‡¶Ü‡¶∞‡ßã ‡¶ñ‡ßÅ‡¶∂‡¶ø      1   
4008  ‡¶∏‡ßá‡¶ñ ‡¶π‡¶æ‡¶∏‡¶ø‡¶®‡¶æ ‡¶ï‡ßã‡¶•‡¶æ‡ßü ‡¶¶‡ßá‡¶∂‡ßá ‡¶®‡ßá‡¶á ‡¶§‡¶æ‡¶∞ ‡¶ï‡¶æ‡¶õ‡ßá ‡¶™‡¶ø‡¶ï‡¶ö‡¶æ‡¶∞ ‡¶¶‡ßá‡¶ñ‡¶æ...      0   
4009                 ‡¶è‡¶á ‡¶ï‡ßç‡¶∑‡¶§‡¶ø ‡¶ú‡¶®‡ßç‡¶Ø ‡¶™‡ßç‡¶∞‡¶ß‡¶æ‡¶®‡¶Æ‡¶®‡ßç‡¶§‡ßç‡¶∞‡ßÄ‡¶∞ ‡¶¶‡¶æ‡¶á ‡•§      0   

                                                premise  
4005                        ‡¶®‡¶æ‡¶∂‡¶ï‡¶§‡¶æ‡¶ï‡¶æ‡¶∞‡ßÄ ‡¶ï‡ßá?‡¶¶‡ßá‡¶∂‡ßá‡¶∞ ‡¶ú‡¶®‡¶ó‡¶£ ‡¶ï‡ßá  
4006                              ‡¶§‡ßã‡¶∞ ‡¶¨‡¶ø‡¶ö‡¶æ‡¶∞ ‡¶Ü‡¶≤‡ßç‡¶≤‡¶æ‡¶π ‡¶ï‡¶∞‡¶¨‡ßá  
4007           ‡¶ï‡¶æ‡¶∞‡ßã ‡¶ö‡ßã‡¶ñ‡ßá‡¶∞ ‡¶™‡¶æ‡¶®‡¶ø ‡¶™‡¶°‡¶º‡ßá‡¶®‡¶ø ‡¶¨‡¶∞‡¶Ç ‡¶ú‡¶®‡¶ó‡¶£ ‡¶Ü‡¶∞‡ßã ‡¶ñ‡ßÅ‡¶∂‡¶ø  
4008  ‡¶∏‡ßá‡¶ñ ‡¶π‡¶æ‡¶∏‡¶ø‡¶®‡¶æ ‡¶ï‡ßã‡¶•‡¶æ‡ßü ‡¶¶‡ßá‡¶∂‡ßá ‡¶®‡ßá‡¶á ‡¶§‡¶æ‡¶∞ ‡¶ï‡¶æ‡¶õ‡ßá ‡¶™‡¶ø‡¶ï‡¶ö‡¶æ‡¶∞ ‡¶¶‡ßá‡¶ñ‡¶æ...  
4009                 ‡¶è‡¶á ‡¶ï‡ßç‡¶∑‡¶§‡¶ø ‡¶ú‡¶®‡ßç‡¶Ø ‡¶™‡ßç‡¶∞‡¶ß‡¶æ‡¶®‡¶Æ‡¶®‡ßç‡¶§‡ßç‡¶∞‡ßÄ‡¶∞ ‡¶¶‡¶æ‡¶á ‡•§

- Îç∞Ïù¥ÌÑ∞ÏÖãÏùÑ ÏÉùÏÑ±Î™®Îç∏Ïùò ÌïôÏäµ ÌòïÏãùÏóê ÎßûÍ≤å Î≥ÄÌòïÌï©ÎãàÎã§.

In [36]:
id_to_label = {0:'neutral', 1:'positive', 2:'negative'}

question_template = "### Human: Îã§Ïùå Îëê Î¨∏Ïû•Ïùò Í¥ÄÍ≥ÑÎ•º entailment, neutral, contradiction Ï§ë ÌïòÎÇòÎ°ú Î∂ÑÎ•òÌï¥Ï§ò. "

train_instructions = [f'{question_template}\npremise: {x}\nhypothesis: {y}\n\n### Assistant: {id_to_label[z]}' for x,y,z in zip(train_data['premise'],train_data['hypothesis'],train_data['label'])]
validation_instructions = [f'{question_template}\npremise: {x}\nhypothesis: {y}\n\n### Assistant: {id_to_label[z]}' for x,y,z in zip(validation_data['premise'],validation_data['hypothesis'],validation_data['label'])]

ds_train = Dataset.from_dict({"text": train_instructions})
ds_validation = Dataset.from_dict({"text": validation_instructions})
instructions_ds_dict = DatasetDict({"train": ds_train, "eval": ds_validation})

In [37]:
instructions_ds_dict['train']['text'][0]

'### Human: Îã§Ïùå Îëê Î¨∏Ïû•Ïùò Í¥ÄÍ≥ÑÎ•º entailment, neutral, contradiction Ï§ë ÌïòÎÇòÎ°ú Î∂ÑÎ•òÌï¥Ï§ò. \npremise: ‡¶§‡ßã‡¶Æ‡¶∞‡¶æ ‡¶∏‡¶¨‡¶æ‡¶á ‡¶è‡¶á ‡¶≠‡¶æ‡¶á ‡¶§‡¶æ‡¶ï‡ßá ‡¶è‡¶ï‡¶ü‡ßÅ  ‡¶ï‡ßã‡¶∞‡ßã‡¶≠‡¶ø‡¶°‡¶ø‡¶ì ‡¶ó‡ßÅ‡¶≤‡ßã ‡¶¶‡ßá‡¶ñ‡ßã ‡¶≠‡¶æ‡¶≤ ‡¶≤‡¶æ‡¶ó‡¶≤‡ßá  ‡¶ï‡ßã‡¶∞‡ßã üôè ‡¶ß‡¶®‡ßç‡¶Ø‡¶¨‡¶æ‡¶¶‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è\nhypothesis: ‡¶§‡ßã‡¶Æ‡¶∞‡¶æ ‡¶∏‡¶¨‡¶æ‡¶á ‡¶è‡¶á ‡¶≠‡¶æ‡¶á ‡¶§‡¶æ‡¶ï‡ßá ‡¶è‡¶ï‡¶ü‡ßÅ  ‡¶ï‡ßã‡¶∞‡ßã‡¶≠‡¶ø‡¶°‡¶ø‡¶ì ‡¶ó‡ßÅ‡¶≤‡ßã ‡¶¶‡ßá‡¶ñ‡ßã ‡¶≠‡¶æ‡¶≤ ‡¶≤‡¶æ‡¶ó‡¶≤‡ßá  ‡¶ï‡ßã‡¶∞‡ßã üôè ‡¶ß‡¶®‡ßç‡¶Ø‡¶¨‡¶æ‡¶¶‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è‚ù§Ô∏è\n\n### Assistant: positive'

## 2. Î™®Îç∏ ÌïôÏäµ

- BeomiÎãòÍªòÏÑú ÌïúÍµ≠Ïñ¥ Îç∞Ïù¥ÌÑ∞ÏÖãÏúºÎ°ú Ï∂îÍ∞Ä ÌõàÎ†® ÌïòÏã† LLaMA2 7B Î™®Îç∏ÏùÑ Fine-tuning ÌïòÍ≤†ÏäµÎãàÎã§.
- Ïö∞ÏÑ†, ÌïôÏäµÏóê ÌïÑÏöîÌïú ArgumentsÎ•º Ï§ÄÎπÑÌï©ÎãàÎã§.

In [38]:
model_name = "beomi/llama-2-ko-7b"


@dataclass
class ScriptArguments:
    model_name: Optional[str] = field(default=model_name, metadata={"help": "the model name"})
    dataset_text_field: Optional[str] = field(default="text", metadata={"help": "the text field of the dataset"})
    log_with: Optional[str] = field(default=None, metadata={"help": "use 'wandb' to log with wandb"})
    learning_rate: Optional[float] = field(default=1.41e-5, metadata={"help": "the learning rate"})
    batch_size: Optional[int] = field(default=4, metadata={"help": "the batch size"})
    seq_length: Optional[int] = field(default=512, metadata={"help": "Input sequence length"})
    gradient_accumulation_steps: Optional[int] = field(
        default=2, metadata={"help": "the number of gradient accumulation steps"}
    )
    load_in_8bit: Optional[bool] = field(default=False, metadata={"help": "load the model in 8 bits precision"})
    load_in_4bit: Optional[bool] = field(default=True, metadata={"help": "load the model in 4 bits precision"})
    use_peft: Optional[bool] = field(default=True, metadata={"help": "Wether to use PEFT or not to train adapters"})
    trust_remote_code: Optional[bool] = field(default=True, metadata={"help": "Enable `trust_remote_code`"})
    output_dir: Optional[str] = field(default="output", metadata={"help": "the output directory"})
    peft_lora_r: Optional[int] = field(default=64, metadata={"help": "the r parameter of the LoRA adapters"})
    peft_lora_alpha: Optional[int] = field(default=16, metadata={"help": "the alpha parameter of the LoRA adapters"})
    logging_steps: Optional[int] = field(default=1, metadata={"help": "the number of logging steps"})
    use_auth_token: Optional[bool] = field(default=False, metadata={"help": "Use HF auth token to access the model"})
    num_train_epochs: Optional[int] = field(default=3, metadata={"help": "the number of training epochs"})
    max_steps: Optional[int] = field(default=-1, metadata={"help": "the number of training steps"})
    save_steps: Optional[int] = field(
        default=100, metadata={"help": "Number of updates steps before two checkpoint saves"}
    )
    save_total_limit: Optional[int] = field(default=10, metadata={"help": "Limits total number of checkpoints."})
    push_to_hub: Optional[bool] = field(default=False, metadata={"help": "Push the model to HF Hub"})
    hub_model_id: Optional[str] = field(default=None, metadata={"help": "The name of the model on HF Hub"})


script_args = ScriptArguments()

- quantizationÏùÑ Ï†ÅÏö©ÌïòÏó¨ Î™®Îç∏ÏùÑ Î∂àÎü¨ÏòµÎãàÎã§.

In [40]:
if script_args.load_in_8bit and script_args.load_in_4bit:
    raise ValueError("You can't load the model in 8 bits and 4 bits at the same time")
elif script_args.load_in_8bit or script_args.load_in_4bit:
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=script_args.load_in_8bit, load_in_4bit=script_args.load_in_4bit
    )
    device_map = {"": 0}
    torch_dtype = torch.bfloat16
else:
    device_map = None
    quantization_config = None
    torch_dtype = None

model = AutoModelForCausalLM.from_pretrained(
    script_args.model_name,
    quantization_config=quantization_config,
    device_map=device_map,
    trust_remote_code=script_args.trust_remote_code,
    torch_dtype=torch_dtype,
    use_auth_token=script_args.use_auth_token,
)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 362.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 21.06 MiB is free. Process 4387 has 14.72 GiB memory in use. Of the allocated memory 14.29 GiB is allocated by PyTorch, and 307.57 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

- LoRAÎ•º Ï†ÅÏö©ÌïòÏó¨ TrainerÎ•º Íµ¨ÏÑ±Ìï©ÎãàÎã§.

In [ ]:
dataset = instructions_ds_dict

training_args = TrainingArguments(
    output_dir=script_args.output_dir,
    per_device_train_batch_size=script_args.batch_size,
    gradient_accumulation_steps=script_args.gradient_accumulation_steps,
    learning_rate=script_args.learning_rate,
    logging_steps=script_args.logging_steps,
    num_train_epochs=script_args.num_train_epochs,
    max_steps=script_args.max_steps,
    report_to=script_args.log_with,
    save_steps=script_args.save_steps,
    save_total_limit=script_args.save_total_limit,
    push_to_hub=script_args.push_to_hub,
    hub_model_id=script_args.hub_model_id,
)

if script_args.use_peft:
    peft_config = LoraConfig(
        r=script_args.peft_lora_r,
        lora_alpha=script_args.peft_lora_alpha,
        bias="none",
        task_type="CAUSAL_LM",
    )
else:
    peft_config = None

trainer = SFTTrainer(
    model=model,
    args=training_args,
    max_seq_length=script_args.seq_length,
    train_dataset=dataset['train'],
    eval_dataset=dataset['eval'],
    dataset_text_field=script_args.dataset_text_field,
    peft_config=peft_config,
)

- ÌïôÏäµÏùÑ ÏßÑÌñâÌï©ÎãàÎã§.
- ColabÏùò V100 Gpu Í∏∞Ï§Ä ÏïΩ 8ÏãúÍ∞ÑÏù¥ ÏÜåÏöîÎê©ÎãàÎã§.

In [ ]:
trainer.train()

trainer.save_model(training_args.output_dir)

- ÌïôÏäµÌïú Î™®Îç∏Ïùò ÏÑ±Îä•ÏùÑ ÌÖåÏä§Ìä∏ Ìï¥ Î¥ÖÎãàÎã§.

In [ ]:
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name)

pipeline = pipeline(
    "text-generation",
    model=model,|
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map={'':0},
)

In [ ]:
query = instructions_ds_dict['eval']['text'][1].split('### Assistant: ')[0] + '### Assistant:'
queries = [instructions_ds_dict['eval']['text'][i].split('### Assistant: ')[0] + '### Assistant:' for i in range(len(instructions_ds_dict['eval']))]
sequences = pipeline(
    queries,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=3,
    early_stopping=True,
    # do_sample=True,
)

In [ ]:
results = []

for seq in sequences:
  result = seq[0]['generated_text'].split('### Assistant:')[1]
  results.append(result)

labels = []

for label in instructions_ds_dict['eval']['text']:
  result = label.split('### Assistant:')[1]
  labels.append(result)

print("Accuracy: ", (len([1 for x, y in zip(results, labels) if y in x]) / len(labels)))